<a href="https://colab.research.google.com/github/Ry02024/24DXsales/blob/main/MachineLearning4TimeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBM

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import keras
from tqdm.autonotebook import tqdm
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from keras.optimizers import RMSprop
import requests
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error  # RMSLE
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import csv
#import lightgbm as lgb
# import optuna.integration.lightgbm as lgb
import random
pd.set_option('display.max_columns', 200)

<ipython-input-2-68312f951660>:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
fdata_dir = "/content/drive/MyDrive/マナビDX/PBL01/演習03（機械学習）/DXQuest_PBL01/Data/FeatureData/"

In [5]:
valid_data_df = pd.read_csv(fdata_dir + 'valid_data_df.csv', index_col=0, header=0)
train_data_df = pd.read_csv(fdata_dir + 'train_data_df.csv', index_col=0, header=0)
test_data_df = pd.read_csv(fdata_dir + 'test_data_df.csv', index_col=0, header=0)

In [6]:
import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [8]:
# ハイパーパラメータを設定
params = {'metric': 'rmse',
          }

drop_list = [
             #'category_id',
             #'main_flag'
]
# データ分割の割合
train_rate_1_11 = 1
validation_rate_1_11 = 0

train_rate_12 = 0.5
validation_rate_12 = 0.5

drop_num=0

n=0
predict_x_df = test_data_df.copy()
# month_target_12_dfをシャッフルル
valid_data_df_shuffled = valid_data_df.sample(frac=1, random_state=n)


# train, validation に2分割
train_num_1_11 = int(train_data_df.shape[0] * train_rate_1_11)
validation_num_1_11 = int(train_data_df.shape[0] * validation_rate_1_11)
train_num_12 = int(valid_data_df_shuffled.shape[0] * train_rate_12)
validation_num_12 = int(valid_data_df_shuffled.shape[0] * validation_rate_12)
train_num_total = train_num_1_11 + train_num_12
validation_num_total = validation_num_1_11 + validation_num_12


train_df_orig = pd.concat([train_data_df.iloc[:train_num_1_11], valid_data_df_shuffled.iloc[:train_num_12]]).sample(frac=1, random_state=n)
validation_df_orig = pd.concat([train_data_df.iloc[train_num_1_11:train_num_1_11+validation_num_1_11],
                                valid_data_df_shuffled.iloc[train_num_12:train_num_12+validation_num_12]]).sample(frac=1, random_state=n)

# 並び順をシャッフル
train_df = train_df_orig.sample(frac=1.0, random_state=n).copy()  # データの5割を使用
validation_df = validation_df_orig.sample(frac=1.0, random_state=n).copy()

# xとyに分割
train_x_df = train_df.drop(['product_price_11', 'product_price_12', 'product_num_11', 'product_num_12'], axis=1)
train_y_df = pd.DataFrame(train_df['product_num_12'])
validation_x_df = validation_df.drop(['product_price_11', 'product_price_12', 'product_num_11', 'product_num_12'], axis=1)
validation_y_df = pd.DataFrame(validation_df['product_num_12'])

# 特徴量を削除
train_x_df = train_x_df.drop(drop_list, axis=1)
validation_x_df = validation_x_df.drop(drop_list, axis=1)
predict_x_df = predict_x_df.drop(drop_list, axis=1)


# 特徴量をランダムでdrop_num個削除
drop_idx = [random.randint(0, 37) for i in range(drop_num)]
drop_columns = train_x_df.columns[drop_idx]
train_x_df = train_x_df.drop(drop_columns, axis=1)
validation_x_df = validation_x_df.drop(drop_columns, axis=1)
predict_x_df = predict_x_df.drop(drop_columns, axis=1)

print('train_x', train_x_df.shape)
print('validation_x', validation_x_df.shape)
print('predict_x', predict_x_df.shape)
# LightGBM用のデータセットに変換
lgb_train = lgb.Dataset(train_x_df, train_y_df)
lgb_eval = lgb.Dataset(validation_x_df, validation_y_df)

callbacks = [
    log_evaluation(period=500),  # 100イテレーションごとに結果を表示
    early_stopping(stopping_rounds=500, verbose=True)
    ]
# 学習
gbm = lgb.train(params,
                lgb_train,
                valid_sets=[lgb_train,lgb_eval],
                num_boost_round=10000,
                callbacks = callbacks,
        )
# 予測
final_predicted = gbm.predict(predict_x_df)

train_x (1788390, 56)
validation_x (1530, 56)
predict_x (3060, 56)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.473893 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9077
[LightGBM] [Info] Number of data points in the train set: 1788390, number of used features: 56
[LightGBM] [Info] Start training from score 0.308211
Training until validation scores don't improve for 500 rounds
[100]	valid_0's rmse: 1.63905
[200]	valid_0's rmse: 1.59505
[300]	valid_0's rmse: 1.56024
[400]	valid_0's rmse: 1.5225
[500]	valid_0's rmse: 1.50227
[600]	valid_0's rmse: 1.49717
[700]	valid_0's rmse: 1.48876
[800]	valid_0's rmse: 1.47818
[900]	valid_0's rmse: 1.47957
[1000]	valid_0's rmse: 1.47366
[1100]	valid_0's rmse: 1.47103
[1200]	valid_0's rmse: 1.46145
[1300]	valid_0's rmse: 1.44612
[1400]	valid_0's rmse: 1.44478
[1500]	valid_0's rmse: 1.43963
[1600]	v

In [9]:
final_predicted

array([2.38167911, 0.55563864, 1.90079572, ..., 0.31000384, 0.18568648,
       0.26725855])

In [10]:
base_dir = '/content/drive/MyDrive/マナビDX/PBL01/演習03（機械学習）/DXQuest_PBL01/Data/'

In [11]:
sub = pd.read_csv(base_dir + 'sample_submission.csv', header=None)

In [12]:
sub[1] = final_predicted

In [13]:
sub

,0,1
0,0,2.381679
1,1,0.555639
2,2,1.900796
3,3,4.564866
4,4,9.841028
...,...,...
3055,3055,2.698795
3056,3056,0.166300
3057,3057,0.310004
3058,3058,0.185686


In [14]:
sub.to_csv(base_dir + 'ID100947_PBL01_ver21.csv', index=False, header=False)